In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.io
import glob
import os
from math import cos,sin,sqrt,pow

In [ ]:
def get_distance_2_pts(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    distance = sqrt((y1-y2)**2+(x2-x1)**2)
    return distance

In [ ]:
# w is the length of the rectangle
def get_4_pts(p1,p2,img):
    x_max,y_max,_ = img.shape
    x1,y1 = p1
    x2,y2 = p2
    alpha = sqrt((y1-y2)**2+(x2-x1)**2)
    
    if alpha > 200 :
        w = -125
    else:
        w = -250
    
    sin_theta = (y1-y2)/alpha
    cos_theta = (x2-x1)/alpha
    x4 = w*sin_theta + x2
    x3 = w*sin_theta + x1
    y4 = w*cos_theta + y2
    y3 = w*cos_theta + y1
    if (x4 > x_max):
        x4 = x_max
    if (x3 > x_max):
        x3 = x_max
    if (y4 > y_max):
        y4 = y_max
    if (y3 > y_max):
        y3 = y_max
    if (x4 < 0):
        x4 = 0
    if (x3 < 0):
        x3 = 0
    if (y4 < 0):
        y4 = 0
    if (y3 < 0):
        y3 = 0
    p3 = [x3,y3]
    p4 = [x4,y4]
    return p1,p2,p3,p4

In [ ]:
def draw_pts_on_img(img,pts,colour):
    copy_img = np.copy(img)
    for p in pts:
        copy_img = cv2.circle(copy_img, tuple(map(round, p)), radius =10, color = colour, thickness = -1)
    return copy_img

In [ ]:
class slot(object):
    def __init__(self,p1,p2,p3,p4,vacancy):
     self.update(p1,p2,p3,p4,vacancy)
    def update(self,p1,p2,p3,p4,vacancy):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.p4 = p4
        self.vacancy = vacancy
        
class slot_img(object):
    def __init__(self,img,mat):
         self.update(img,mat)
    def update(self,img,mat):
        colours = [(255,0,0),(0,255,0),(0,0,255)]
        colour_picker = 0
        self.img = img
        self.img_anot = np.copy(img)
        self.mat = mat
        self.slots_num = len(mat['slots'])
        self.slots = []
        for s in mat['slots']:
            p1_num = s[0] - 1
            p2_num = s[1] - 1
            p1 = mat['marks'][p1_num]
            p2 = mat['marks'][p2_num]
            pts = get_4_pts(p1,p2,img)
            _,_,p3,p4 = pts
            vacancy_num = s[2]
            vacancy = 'non-vacant'
            if (vacancy_num == 0):
                vacancy = 'vacant'
            elif (vacancy_num == 1):
                vacancy = 'non-vacant'
            elif (vacancy_num == 2):
                vacancy = 'parking'
            self.slots.append(slot(p1,p2,p3,p4,vacancy))
            colour = colours[colour_picker]
            
            self.img_anot = draw_pts_on_img(self.img_anot,pts,colour)
            colour_picker += 1
            colour_picker %= 3

In [ ]:
slots = []
iterator = 0
num_samples = 50
training_dir = 'training/'
for file in glob.glob(training_dir + '*.jpg'):
    file_name = file[:-4]
    img = cv2.imread(file)
    mat = scipy.io.loadmat(file_name + '.mat')
    s = slot_img(img,mat)
    slots.append(s)
    iterator += 1
    if iterator >= num_samples:
        break

In [ ]:
for s in slots:
    plt.imshow(s.img_anot)
    plt.show()
    for slot in s.slots:
        p1 = slot.p1
        p2 = slot.p2
        p3 = slot.p3
        p4 = slot.p4
        pts_src = np.array([p1,p2,p3,p4],np.float32)
        width = get_distance_2_pts(p1,p2)
        height = get_distance_2_pts(p4,p2)
        pts_dst = np.array([[0.0,0.0],[width, 0.0],[ 0.0,height],[width,height]],np.float32)
        m_warp = cv2.getPerspectiveTransform(pts_src, pts_dst)
        warp_img = cv2.warpPerspective(s.img, m_warp, (int(width), int(height)))
        plt.imshow(warp_img)
        plt.show()
        print(slot.vacancy)